In [ ]:
#Instalando o stumpy no ambiente do colab
!pip install stumpy

In [1]:
#importando as libs que serão utilizadas
import pandas as pd
import numpy as np

import stumpy
from stumpy.floss import _cac
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, FancyArrowPatch
from matplotlib import animation
from IPython.display import HTML
import os

In [14]:
#carregando dado que será utilizado
df = pd.read_csv("https://zenodo.org/record/4276400/files/Semantic_Segmentation_TiltABP.csv?download=1")
df.head()

,time,abp
0,0,6832.0
1,1,6928.0
2,2,6968.0
3,3,6992.0
4,4,6980.0


In [36]:
#plotando dado
px.line(df['abp'])

In [60]:
start = 25000 - 2500
stop = 25000 + 2500
data = df.iloc[start:stop, 1]

In [89]:
#primeiramente escolhe o tamanho da janela
window = 210
#agora é necessario calcular a matrix profile
mp = stumpy.stump(data, m=window)

In [66]:
#com a matrix profile calculada, basta chamar a função fluss, que recebe como parametro os indices da matrix profile
#Uma aproximação do tamanho da subsequencia L
#numero de regiões a serem procuradas
#e fator de exclusão, para anular o inicio e fim da curva arco, que de acordo com artigo entre 1 e 5 é razoavel
cac, regime_locations = stumpy.fluss(mp[:, 1], L=window, n_regimes=2, excl_factor=1)

In [67]:
#plot do sinal segmentado
fig = make_subplots(rows=2, cols=1)
fig.add_trace(go.Scatter(y=data, name='Data'), row=1, col=1)
for i in regime_locations:
  fig.add_vline(x=i)
fig.add_trace(go.Scatter(y=cac, name='CAC'), row=2, col=1)
for i in regime_locations:
  fig.add_vline(x=i)
fig.show()

In [68]:
data = df['abp']

In [69]:
window = 210
mp = stumpy.stump(data, m=window)

In [78]:
cac, regime_locations = stumpy.fluss(mp[:, 1], L=400, n_regimes=5, excl_factor=2)

In [79]:
#plot do sinal segmentado
fig = make_subplots(rows=2, cols=1)
fig.add_trace(go.Scatter(y=data, name='Data'), row=1, col=1)
for i in regime_locations:
  fig.add_vline(x=i)
fig.add_trace(go.Scatter(y=cac, name='CAC'), row=2, col=1)
for i in regime_locations:
  fig.add_vline(x=i)
fig.show()

In [91]:
#Calcula a matrix profile do dado antigo
window = 210
old_data = df.iloc[20000:20000+5000, 1].values  # This is well before the regime change has occurred

mp = stumpy.stump(old_data, m=window)

In [ ]:
#calcula a curva cac para uma demonstração
cac_1d = _cac(mp[:, 3], window, bidirectional=False, excl_factor=1)  # This is for demo purposes only. Use floss() below!

In [93]:
new_data = df.iloc[25000:25000+5000, 1].values

In [94]:
stream = stumpy.floss(mp, old_data, m=window, L=window, excl_factor=1)

In [95]:
windows = []
for i, t in enumerate(new_data):
    stream.update(t)

    if i % 100 == 0:
        windows.append((stream.T_, stream.cac_1d_))

In [96]:
fig, axs = plt.subplots(2, sharex=True, gridspec_kw={'hspace': 0})

axs[0].set_xlim((0, mp.shape[0]))
axs[0].set_ylim((-0.1, max(np.max(old_data), np.max(new_data))))
axs[1].set_xlim((0, mp.shape[0]))
axs[1].set_ylim((-0.1, 1.1))

lines = []
for ax in axs:
    line, = ax.plot([], [], lw=2)
    lines.append(line)
line, = axs[1].plot([], [], lw=2)
lines.append(line)

def init():
    for line in lines:
        line.set_data([], [])
    return lines

def animate(window):
    data_out, cac_out = window
    for line, data in zip(lines, [data_out, cac_out, cac_1d]):
        line.set_data(np.arange(data.shape[0]), data)
    return lines

anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=windows, interval=100,
                               blit=True)

anim_out = anim.to_jshtml()
plt.close()  # Prevents duplicate image from displaying
if os.path.exists("None0000000.png"):
    os.remove("None0000000.png")  # Delete rogue temp file

HTML(anim_out)